[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/openai/fine-tuning/gpt-3.5-agent-training/00-fine-tuning.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/openai/fine-tuning/gpt-3.5-agent-training/00-fine-tuning.ipynb)

# Fine-tuning GPT-3.5 with a retrieval tool

This notebook walks through fine-tuning GPT-3.5 Turbo on conversations that use a Pinecone-backed vector search tool. You will load a dataset of tool-using conversations, run a fine-tuning job with the OpenAI API, then use the fine-tuned model with LangChain and Pinecone.

## Prerequisites

- Python with `datasets`, `langchain`, `pinecone`, and `openai` (install in the next cell).
- [OpenAI API key](https://platform.openai.com/api-keys) for fine-tuning and inference.
- [Pinecone API key](https://app.pinecone.io/) for the vector search tool used by the fine-tuned model.

In [ ]:
import requests

!pip install -qU datasets==2.14.4 langchain==0.0.274 pinecone==8.0.0 openai==0.27.9 requests==2.32.3

res = requests.get(
    "https://raw.githubusercontent.com/pinecone-io/examples/master/learn/generation/openai/fine-tuning/gpt-3.5-agent-training/chains.py"
)
with open("chains.py", "w") as fp:
    fp.write(res.text)

import os
from getpass import getpass
from time import sleep

import openai
from chains import VectorDBChain
from datasets import load_dataset
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
# Setup complete (imports and chains.py in previous cell)

In [ ]:
data = load_dataset("jamescalam/agent-conversations-retrieval-tool", split="train")
data

In [ ]:
data["messages"][0]

In [ ]:
data.to_json("conversations.jsonl")

## Running Training

First we upload the files:

In [ ]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or "YOUR_API_KEY"
openai.api_key = os.environ["OPENAI_API_KEY"]


res = openai.File.create(file=open("conversations.jsonl", "r"), purpose="fine-tune")
res

In [ ]:
file_id = res["id"]
file_id

We then create the fine-tuning job _(note, it can take some time before the file above is ready)_.

In [ ]:
res = openai.FineTuningJob.create(training_file=file_id, model="gpt-3.5-turbo")
res

In [ ]:
job_id = res["id"]
job_id

We can retrieve info for a our fine-tuning job like so:

In [ ]:
openai.FineTuningJob.retrieve(job_id)

The `"finished_at"` value is still `null`, so fine-tuning isn't yet complete. We can check for events from our fine-tuning job while we wait:

In [ ]:
openai.FineTuningJob.list_events(id=job_id)

We can setup a check for fine-tuning completion (or wait for OpenAI to send you an email telling you that the job has completed):

In [ ]:
while True:
    res = openai.FineTuningJob.retrieve(job_id)
    if res["finished_at"] is not None:
        break
    else:
        print(".", end="")
        sleep(100)

Once complete, we can see our model details in the `res`:

In [ ]:
res

We access our fine-tuned model name:

In [ ]:
ft_model = res["fine_tuned_model"]
ft_model

Finally, we use our new model!

In [ ]:
ft_model = "ft:gpt-3.5-turbo-0613:pinecone::7s8gnk9R"

In [ ]:
llm = ChatOpenAI(temperature=0.5, model_name=ft_model)

memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True, output_key="output"
)
pinecone_api_key = os.environ.get("PINECONE_API_KEY") or getpass(
    "Enter your Pinecone API key: "
)
vdb = VectorDBChain(
    index_name="llama-2-arxiv-papers",
    environment=os.getenv("PINECONE_ENV") or "us-east-1",
    pinecone_api_key=pinecone_api_key,
)

vdb_tool = Tool(
    name=vdb.name,
    func=vdb.query,
    description="This tool allows you to get research information about LLMs.",
)

In [ ]:
agent = initialize_agent(
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools=[vdb_tool],
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method="generate",
    memory=memory,
    return_intermediate_steps=True,
)

In [ ]:
agent("tell me about Llama 2?")

In [ ]:
agent("what makes llama 2 so special?")

In [ ]:
agent("tell me about llama 2 red teaming?")

---